In [6]:
from queue import PriorityQueue

# define the 8-puzzle problem
initial_state = [[0,1,3],[4,2,5],[7,8,6]]
goal_state = [[1,2,3],[4,5,6],[7,8,0]]

# define the Manhattan distance heuristic function
def manhattan_distance(state):
    distance = 0
    for i in range(3):
        for j in range(3):
            if state[i][j] != 0:
                print(" State[i][j]",state[i][j])
                row = (state[i][j] - 1) // 3
                col = (state[i][j] - 1) % 3
                print("row",row)
                print("col",col)
                distance += abs(i - row) + abs(j - col)
                print("distance",distance)
    return distance

# define the A* search algorithm
def a_star(initial_state, goal_state):
    frontier = PriorityQueue()
    explored = set()
    frontier.put((0, initial_state, []))

    while not frontier.empty():
        cost, current_state, path = frontier.get()
        print("frontier:", frontier.queue)
        if current_state == goal_state:
            print("Solution found!",current_state)
            return path
        explored.add(tuple(map(tuple, current_state)))
        zero_row, zero_col = [(i, j) for i in range(3) for j in range(3) if current_state[i][j] == 0][0]

        # generate child states
        for row, col in [(zero_row-1, zero_col), (zero_row+1, zero_col), (zero_row, zero_col-1), (zero_row, zero_col+1)]:
            if row >= 0 and row < 3 and col >= 0 and col < 3:
                child_state = [list(row) for row in current_state]
                child_state[zero_row][zero_col], child_state[row][col] = child_state[row][col], child_state[zero_row][zero_col]
                print(child_state)
                if tuple(map(tuple, child_state)) not in explored:
                    priority = len(path) + 1 + manhattan_distance(child_state)
                    frontier.put((priority, child_state, path + [(row, col)]))

    return None

# solve the 8-puzzle problem
print('Initial state:', initial_state)
solution = a_star(initial_state, goal_state)
if solution:
    print(f'Solution found in {len(solution)} steps: {solution}')
else:
    print('No solution found')


Initial state: [[0, 1, 3], [4, 2, 5], [7, 8, 6]]
frontier: []
[[4, 1, 3], [0, 2, 5], [7, 8, 6]]
 State[i][j] 4
row 1
col 0
distance 1
 State[i][j] 1
row 0
col 0
distance 2
 State[i][j] 3
row 0
col 2
distance 2
 State[i][j] 2
row 0
col 1
distance 3
 State[i][j] 5
row 1
col 1
distance 4
 State[i][j] 7
row 2
col 0
distance 4
 State[i][j] 8
row 2
col 1
distance 4
 State[i][j] 6
row 1
col 2
distance 5
[[1, 0, 3], [4, 2, 5], [7, 8, 6]]
 State[i][j] 1
row 0
col 0
distance 0
 State[i][j] 3
row 0
col 2
distance 0
 State[i][j] 4
row 1
col 0
distance 0
 State[i][j] 2
row 0
col 1
distance 1
 State[i][j] 5
row 1
col 1
distance 2
 State[i][j] 7
row 2
col 0
distance 2
 State[i][j] 8
row 2
col 1
distance 2
 State[i][j] 6
row 1
col 2
distance 3
frontier: [(6, [[4, 1, 3], [0, 2, 5], [7, 8, 6]], [(1, 0)])]
[[1, 2, 3], [4, 0, 5], [7, 8, 6]]
 State[i][j] 1
row 0
col 0
distance 0
 State[i][j] 2
row 0
col 1
distance 0
 State[i][j] 3
row 0
col 2
distance 0
 State[i][j] 4
row 1
col 0
distance 0
 State[i][j] 5


### another approach same problem

In [7]:
from queue import PriorityQueue
import numpy as np

# Define the goal state
goal_state = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

# Define the initial state
initial_state = np.array([[0,1,3],[4,2,5],[7,8,6]])

# Define the size of the puzzle
size = initial_state.shape[0]

# Define the helper function to find the position of a given value in the state
def find_position(state, value):
    x, y = np.where(state == value)
    return x[0], y[0]

# Define the Manhattan distance heuristic
def manhattan_distance(state):
    distance = 0
    for i in range(size):
        for j in range(size):
            value = state[i, j]
            if value != 0:
                x_goal, y_goal = find_position(goal_state, value)
                x, y = find_position(state, value)
                distance += abs(x - x_goal) + abs(y - y_goal)
    return distance

# Define the Node class
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.g = 0
        self.h = manhattan_distance(state)
        if parent is not None:
            self.g = parent.g + 1

    def __lt__(self, other):
        return self.g + self.h < other.g + other.h

    def get_path(self):
        path = [self.state]
        node = self.parent
        while node is not None:
            path.append(node.state)
            node = node.parent
        path.reverse()
        return path

# Define the A* search function
def astar_search(initial_state):
    # Create the initial node
    initial_node = Node(initial_state)

    # Create the priority queue and add the initial node
    queue = PriorityQueue()
    queue.put(initial_node)

    # Create the set of visited states
    visited = set()

    # Search loop
    while not queue.empty():
        # Get the node with the lowest f score
        node = queue.get()

        # Check if the goal state is reached
        if np.array_equal(node.state, goal_state):
            return node.get_path()

        # Generate the children of the node
        children = generate_children(node.state)

        # Add the children to the priority queue if they haven't been visited
        for child in children:
            if tuple(child.flatten()) not in visited:
                visited.add(tuple(child.flatten()))
                child_node = Node(child, node)
                queue.put(child_node)
    # No solution found
    return None

# Define the generate_children function
def generate_children(state):
    children = []
    x, y = np.where(state == 0)
    if x > 0:
        child = state.copy()
        child[x, y] = state[x - 1, y]
        child[x - 1, y] = 0
        children.append(child)
    if x < size - 1:
        child = state.copy()
        child[x, y] = state[x + 1, y]
        child[x + 1, y] = 0
        children.append(child)
    if y > 0:
        child = state.copy()
        child[x, y] = state[x, y - 1]
        child[x, y - 1] = 0
        children.append(child)
    if y < size - 1:
        child = state.copy()
        child[x, y] = state[x, y + 1]
        child[x, y + 1] = 0
        children.append(child)
    return children

# Solve the puzzle 
solution = astar_search(initial_state)
if solution is not None:
    print(f'Solution found in {len(solution) - 1} steps:')
    for state in solution:
        print(state)
else:
    print('No solution found')
    


Solution found in 4 steps:
[[0 1 3]
 [4 2 5]
 [7 8 6]]
[[1 0 3]
 [4 2 5]
 [7 8 6]]
[[1 2 3]
 [4 0 5]
 [7 8 6]]
[[1 2 3]
 [4 5 0]
 [7 8 6]]
[[1 2 3]
 [4 5 6]
 [7 8 0]]
